In [2]:
!pip install pyro-ppl

import torch
import torch.nn as nn
import torch.autograd as autograd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import pyro
import pyro.distributions as dist
from pyro.nn import PyroModule, PyroSample
from pyro.infer import MCMC, NUTS, Predictive

/home/soumik/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:

#device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
print(f'device: {device}')

device: cpu


In [15]:
pyro.set_rng_seed(42)

class BayesianNN(PyroModule):
    def __init__(self):
        super().__init__()
        self.layer1 = PyroModule[nn.Linear](1,5)
        self.layer1.weight = PyroSample(dist.Normal(0.,1.).expand([5,1]).to_event(2))
        self.layer1.bias = PyroSample(dist.Normal(0.,1.).expand([5]).to_event(1))

        self.layer2 = PyroModule[nn.Linear](5,5)
        self.layer2.weight = PyroSample(dist.Normal(0.,1.).expand([5,5]).to_event(2))
        self.layer2.bias = PyroSample(dist.Normal(0.,1.).expand([5]).to_event(1))

        self.layer3 = PyroModule[nn.Linear](5,1)
        self.layer3.weight = PyroSample(dist.Normal(0.,1.).expand([1,5]).to_event(2))
        self.layer3.bias = PyroSample(dist.Normal(0.,1.).expand([1]).to_event(1))

        self.activation = nn.Tanh()

    def forward(self,t):
        t = t.view(-1,1)

        h1 = self.activation(self.layer1(t))
        h2 = self.activation(self.layer2(h1))

        u_pred = self.layer3(h2).squeeze(-1)

        return u_pred

     


In [16]:
# Generate Collocation Points

N_f = 1000
t_f = torch.linspace(0,5,N_f).view(-1,1).to(device).requires_grad_(True)

# Generate Initial Points

t_ic = torch.tensor([0.0], requires_grad=True).to(device)

In [17]:
# Generate synthetic sensor data

N_obs = 10
t_obs = torch.linspace(0, 5, N_obs).unsqueeze(-1).to(device)
u_true = (t_obs**2) / 2 + t_obs
noise_std = 0.1
u_obs = u_true + noise_std * torch.randn_like(u_true)

In [20]:
# Physics Likelihood

model =  BayesianNN()

def physics(t_f,t_ic,lambda_phy = 100.0, lambda_ic = 100.0):
    #model = BayesianNN()
    # u = model(t_f)
    # du_dt = torch.autograd.grad(u,t_f,grad_outputs=torch.ones_like(u),create_graph=True)[0]

    # res_pde = du_dt - t_f

    # loss_pde = torch.mean(res_pde**2)

    # pyro.factor('physics loss', -lambda_phy * loss_pde)

    # Sensor Likelihood

    u_pred_obs = model(t_obs)
    pyro.sample("obs", dist.Normal(u_pred_obs.squeeze(-1), noise_std).to_event(1),obs=u_obs.squeeze(-1))


    # u_ic = model(t_ic)

    # res_ic = u_ic - 0.0

    # loss_ic = torch.mean(res_ic ** 2)

    # pyro.factor('initial condition loss', -lambda_ic * loss_ic)


    

In [21]:

nuts_kernel = NUTS(physics)
mcmc = MCMC(nuts_kernel, num_samples=100, warmup_steps=50)
mcmc.run(t_f, t_ic)


samples = mcmc.get_samples()

Sample: 100%|██████████| 150/150 [02:22,  1.05it/s, step size=3.01e-03, acc. prob=0.929]


In [22]:
# Extract posterior samples

samples = mcmc.get_samples()